In [1]:
base_dir='dataset/annotations/annotations/tcia-lidc-xml/'

In [2]:
import os

c = 0
# Verify XML file paths
for root_dir, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".xml"):
            c += 1
            # print(os.path.join(root_dir, file))  # Print XML file paths
print(c)


1319


In [4]:
import xml.etree.ElementTree as ET

file_path='dataset/annotations/annotations/tcia-lidc-xml/157/158.xml'

# Parse the XML file
tree = ET.parse(file_path)
root = tree.getroot()

# Print the full XML structure
print(ET.tostring(root, encoding="utf-8").decode("utf-8"))

<ns0:LidcReadMessage xmlns:ns0="http://www.nih.gov" uid="1.3.6.1.4.1.14519.5.2.1.6279.6001.1308168927827.0">
 <ns0:ResponseHeader>
  <ns0:Version>1.7</ns0:Version>
  <ns0:MessageId>1151420</ns0:MessageId>
  <ns0:DateRequest>2006-06-05</ns0:DateRequest>
  <ns0:TimeRequest>17:05:26</ns0:TimeRequest>
  <ns0:RequestingSite>removed</ns0:RequestingSite>
  <ns0:ServicingSite>removed</ns0:ServicingSite>
  <ns0:TaskDescription>Second unblinded read</ns0:TaskDescription>
  <ns0:CtImageFile>removed</ns0:CtImageFile>
  <ns0:SeriesInstanceUid>1.3.6.1.4.1.14519.5.2.1.6279.6001.303494235102183795724852353824</ns0:SeriesInstanceUid>
  <ns0:StudyInstanceUID>1.3.6.1.4.1.14519.5.2.1.6279.6001.339170810277323131167631068432</ns0:StudyInstanceUID>
  <ns0:DateService>2006-06-05</ns0:DateService>
  <ns0:TimeService>17:05:26</ns0:TimeService>
  <ns0:ResponseDescription>1 - Reading complete</ns0:ResponseDescription>
  <ns0:ResponseComments>Merged, reader anonymized, unblinded responses</ns0:ResponseComments>
 

In [20]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Function to parse an XML file
def parse_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        print(f"\n✅ Successfully parsed: {file_path}")
        print(
            ET.tostring(root, encoding="utf8").decode("utf8")[:500]
        )  # Print first 500 chars
    except Exception as e:
        print(f"\n❌ Error parsing: {file_path} -> {e}")
    
    nodule_data = []
    
    for reading_session in root.iter():
        if "readingSession" in reading_session.tag.lower():
            print(f"🟢 Found readingSession in {file_path}")
            for unblinded_nodule in reading_session.iter():
                if "unblindedReadNodule" in unblinded_nodule.tag.lower():
                    print(f"🟢 Found unblindedReadNodule in {file_path}")
                    
                    # Extract Nodule ID
                    nodule_id = None
                    for child in unblinded_nodule.iter():
                        if "noduleID" in child.tag.lower():
                            nodule_id = child.text.strip() if child.text else "Missing"
                            break
                    nodule_id = nodule_id if nodule_id else "Missing"

                    # Extract Malignancy
                    malignancy = None
                    for child in unblinded_nodule.iter():
                        if "malignancy" in child.tag.lower():
                            malignancy = child.text.strip() if child.text else "Missing"
                            break
                    malignancy = malignancy if malignancy else "Missing"

                    # Extract Bounding Box
                    x_coords = []
                    y_coords = []
                    
                    for edge in unblinded_nodule.iter():
                        if "edgeMap" in edge.tag.lower():
                            x = edge.find("xCoord")
                            y = edge.find("yCoord")
                            if x is not None and y is not None:
                                x_coords.append(int(x.text))
                                y_coords.append(int(y.text))

                    if x_coords and y_coords and len(x_coords) > 1:
                        bbox = [min(x_coords), min(y_coords), max(x_coords), max(y_coords)]
                    else:
                        bbox = "Invalid"
                        print(f"⚠️ EdgeMap missing in file: {file_path}")

                    # Append results
                    nodule_data.append([file_path, nodule_id, malignancy, bbox])

    return nodule_data

# Directory containing XML files
base_dir = "dataset/annotations/annotations/tcia-lidc-xml"

# Collect data from all XML files
all_nodule_data = []
xml_count = 0

for root_dir, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".xml"):
            file_path = os.path.join(root_dir, file)
            xml_count += 1
            print(f"Processing: {file_path}")
            all_nodule_data.extend(parse_xml(file_path))

print(f"Total XML files processed: {xml_count}")

# Convert to DataFrame
df = pd.DataFrame(all_nodule_data, columns=["File", "NoduleID", "Malignancy", "BoundingBox"])
print(df.head())
print(df.shape)


Processing: dataset/annotations/annotations/tcia-lidc-xml\161-resubmitted-correction-3-9-12.xml

✅ Successfully parsed: dataset/annotations/annotations/tcia-lidc-xml\161-resubmitted-correction-3-9-12.xml
<?xml version='1.0' encoding='utf8'?>
<ns0:LidcReadMessage xmlns:ns0="http://www.nih.gov" uid="1.3.6.1.4.1.14519.5.2.1.6279.6001.1308168927148.0">
 <ns0:ResponseHeader>
  <ns0:Version>1.7</ns0:Version>
  <ns0:MessageId>1152727</ns0:MessageId>
  <ns0:DateRequest>2006-06-05</ns0:DateRequest>
  <ns0:TimeRequest>17:08:37</ns0:TimeRequest>
  <ns0:RequestingSite>removed</ns0:RequestingSite>
  <ns0:ServicingSite>removed</ns0:ServicingSite>
  <ns0:TaskDescription>Second unblinded read</ns0:TaskDescripti
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\158.xml

✅ Successfully parsed: dataset/annotations/annotations/tcia-lidc-xml\157\158.xml
<?xml version='1.0' encoding='utf8'?>
<ns0:LidcReadMessage xmlns:ns0="http://www.nih.gov" uid="1.3.6.1.4.1.14519.5.2.1.6279.6001.1308168927827.

In [21]:
df.head

<bound method NDFrame.head of Empty DataFrame
Columns: [File, NoduleID, Malignancy, BoundingBox]
Index: []>

In [22]:
import xml.etree.ElementTree as ET
import os

NS = {"nih": "http://www.nih.gov"}  # Define namespace


def parse_xml(xml_file):
    if not os.path.exists(xml_file):
        print(f"Error: {xml_file} not found.")
        return None

    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = []

    # Loop through all reading sessions (radiologists' annotations)
    for read_session in root.findall("nih:readingSession", NS):
        rad_id = read_session.find("nih:servicingRadiologistID", NS)
        rad_id = rad_id.text if rad_id is not None else "Unknown"

        # Find all nodules
        for nodule in read_session.findall("nih:unblindedReadNodule", NS):
            nodule_id = nodule.find("nih:noduleID", NS)
            nodule_id = nodule_id.text if nodule_id is not None else "N/A"

            malignancy = nodule.find("nih:characteristics/nih:malignancy", NS)
            malignancy = (
                int(malignancy.text) if malignancy is not None else -1
            )  # -1 means missing value

            # Extract bounding box (if available)
            roi = nodule.findall("nih:roi", NS)
            if roi:
                for r in roi:
                    edge_maps = r.findall("nih:edgeMap", NS)
                    bbox = []
                    for edge in edge_maps:
                        x = int(edge.find("nih:xCoord", NS).text)
                        y = int(edge.find("nih:yCoord", NS).text)
                        bbox.append((x, y))

                    annotations.append(
                        {
                            "File": os.path.basename(xml_file),
                            "NoduleID": nodule_id,
                            "Malignancy": malignancy,
                            "BoundingBox": bbox,
                        }
                    )

    return annotations


# Example Usage
xml_file_path = "dataset/annotations/annotations/tcia-lidc-xml/157/158.xml"
annotations = parse_xml(xml_file_path)
print(annotations)  # Check if data is extracted properly


[{'File': '158.xml', 'NoduleID': '0', 'Malignancy': -1, 'BoundingBox': [(177, 353)]}, {'File': '158.xml', 'NoduleID': '3', 'Malignancy': 3, 'BoundingBox': [(364, 172), (364, 171), (364, 170), (365, 169), (365, 168), (365, 167), (365, 166), (365, 165), (365, 164), (365, 163), (364, 164), (363, 164), (362, 165), (361, 166), (360, 167), (359, 168), (359, 169), (358, 170), (359, 171), (360, 171), (361, 170), (362, 169), (363, 169), (363, 171), (364, 172)]}, {'File': '158.xml', 'NoduleID': '3', 'Malignancy': 3, 'BoundingBox': [(352, 183), (353, 182), (354, 181), (355, 181), (356, 181), (357, 181), (358, 181), (359, 181), (360, 181), (361, 181), (362, 180), (363, 180), (364, 180), (365, 179), (366, 178), (367, 178), (367, 177), (368, 176), (368, 175), (369, 174), (369, 173), (369, 172), (369, 171), (370, 170), (371, 169), (372, 169), (373, 168), (373, 167), (372, 166), (372, 165), (371, 164), (370, 163), (369, 162), (368, 161), (367, 161), (366, 161), (365, 161), (364, 161), (363, 161), (362